<a href="https://colab.research.google.com/github/BarGinger/DIS-Assignment/blob/main/Src/dis_notebook_02_11_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install graphframes
!pip install sparkmeasure==0.24
!pip install matplotlib seaborn
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u422-b05-1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u422-b05-1~22.04) ...
Sel

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col,
    udf,
    row_number,
    countDistinct,
    collect_list,
    struct,
    count,
    sum,
    avg,
    expr,
    percentile_approx,
    max as spark_max,
    explode,
    round,
    rand,
    monotonically_increasing_id,
    array,
    lit,
    broadcast,
    lag
)
import pyspark.sql.functions as F
from sparkmeasure import StageMetrics
from pyspark.sql.types import (
    StringType, IntegerType, BinaryType, DoubleType,
    ArrayType, StructType, StructField, LongType, TimestampType
)
from pyspark.sql import Window
from datetime import datetime, timedelta
from graphframes import GraphFrame
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
import pandas as pd
import pickle
import base64
from sparkmeasure import StageMetrics # for resources monitoring
from functools import wraps
import time
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from operator import truediv
from google.colab import files

In [4]:
# Monitor CPU, Memory and running time
def format_memory(report_memory_output):
  # Initialize a list to store parsed data
    parsed_data = []

    # Iterate through each line in report_memory_output and parse data
    for line in report_memory_output.split('\n'):
        if line.startswith("Stage"):
            # Split the line into parts
            parts = line.split()

            # Extract information
            stage = parts[1]
            metric = parts[2]
            raw_value = int(parts[6])  # Raw value in bytes (integer)

            # Extract formatted value and units
            formatted_value_with_units = " ".join(parts[7:]).replace("(", "").replace(")", "")
            formatted_value, units = formatted_value_with_units.split(" ", 1)

            # Append the extracted information to parsed_data
            parsed_data.append({
                "stageId": stage,
                "memory_metric": metric,
                "memory_raw_value_bytes": raw_value,
                "memory_formatted_value": formatted_value,
                "memory_units": units
            })

    # Create DataFrame
    df = pd.DataFrame(parsed_data)
    return spark.createDataFrame(df)


def track_stage(stage_name):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"Starting {stage_name}")
            stagemetrics.begin()  # Begin collecting metrics for this stage

            result = func(*args, **kwargs)  # Run the actual function

            stagemetrics.end()  # Stop collecting metrics for this stage

            time.sleep(15)

            # Generate metrics DataFrame
            print(f"Completed {stage_name}\n")
            df_metrics_all = stagemetrics.create_stagemetrics_DF()
            df_metrics_agg = stagemetrics.aggregate_stagemetrics_DF()
            # Add stage_name column and join metrics and memory DataFrames
            df_metrics_agg = df_metrics_agg.withColumn("stage_name", pyspark.sql.functions.lit(stage_name))
            df_metrics_agg = df_metrics_agg.withColumn("dataset", pyspark.sql.functions.lit(dataset_file_path))

            df_metrics_agg.show(truncate=False)


            # Generate memory information DataFrame
            # memory_info = stagemetrics.report_memory()
            # df_memory = format_memory(memory_info)


            # df_metrics = df_metrics.join(df_memory, on=['stageId'], how='left')

            # Set write mode based on the stage
            if "Stage 1" in stage_name and clear_csv:
                write_mode = "overwrite"
                header = "true"
            else:
                write_mode = "append"
                header = "true"

            # Write metrics to CSV with appropriate mode and header settings
            df_metrics_agg.coalesce(1).write \
                .mode(write_mode) \
                .option("header", header) \
                .csv(f"{dataset_name}_stage_metrics")

            return result
        return wrapper
    return decorator

In [5]:
# Utils functions

# Convert YYMMDDHHMM to a proper datetime object
def calculate_duration_minutes(start_time, end_time):
  """
  Calculate the duration between two times in minutes.

  Parameters:
  -----------
  start_time : str
      The start time in HH:MM:SS format.
  end_time : str
      The end time in HH:MM:SS format.

  Returns:
  --------
  duration_minutes : float
      The duration between start_time and end_time in minutes.
  """
  start_datetime = convert_to_datetime(start_time)
  end_datetime = convert_to_datetime(end_time)
  duration = end_datetime - start_datetime
  duration_minutes = duration.total_seconds() / 60
  return duration_minutes

def convert_to_datetime(time_str):
  """
  Convert a time string in '%y%m%d%H%M' format to a datetime object.

  Parameters:
  -----------
  time_str : str
      The time string in '%y%m%d%H%M' format.

  Returns:
  --------
  datetime_obj : datetime.datetime
      The datetime object representing the given time string.
  """
  # Use datetime.datetime.strptime to parse the time string
  # This is the correct way to use strptime, avoiding the AttributeError
  return datetime.strptime(str(time_str), '%y%m%d%H%M')

# Define UDF for calculating duration in DDHHMM format
def calculate_duration_string(start_time, end_time):
    start_dt = convert_to_datetime(start_time)
    end_dt = convert_to_datetime(end_time)
    duration = end_dt - start_dt

    days = duration.days
    hours, remainder = divmod(duration.seconds, 3600)
    minutes = remainder // 60
    return f'{days:02d}{hours:02d}{minutes:02d}'

# prompt: print csr_matrix_result pretty
def pretty_print_csr_matrix(csr_matrix_result):
  """Prints a CSR matrix in a readable format."""

  rows, cols = csr_matrix_result.nonzero()
  data = csr_matrix_result.data

  df = pd.DataFrame({
      'Row': rows,
      'Col': cols,
      'Value': data
  })

  print(df)

def create_csr_matrix_from_edges_with_spark(members_df):
    """
    Creates a CSR matrix from a Spark DataFrame based on unique vertices.

    Args:
        members_df: Spark DataFrame with 'community_id' and 'members' columns.

    Returns:
        A CSR matrix.
    """

    # Explode the members array to get each connection in separate rows
    exploded_df = members_df.select(
        "community_id",
        explode("members").alias("member")
    ).select(
        "community_id",
        col("member.Client1").alias("Client1"),
        col("member.Client2").alias("Client2"),
        col("member.duration_minutes").alias("duration_minutes")
    )

    # Get unique clients and create a mapping to indices
    unique_clients = exploded_df.select("Client1").union(exploded_df.select("Client2")).distinct().rdd.flatMap(lambda x: x).collect()
    client_to_index = {client: i for i, client in enumerate(unique_clients)}
    num_clients = len(unique_clients)

    # Extract data for CSR matrix
    rows = exploded_df.select("Client1").rdd.map(lambda row: client_to_index[row[0]]).collect()
    cols = exploded_df.select("Client2").rdd.map(lambda row: client_to_index[row[0]]).collect()
    data = exploded_df.select("duration_minutes").rdd.flatMap(lambda x: x).collect()

    # Create CSR matrix
    csr = csr_matrix((data, (rows, cols)), shape=(num_clients, num_clients))

    return csr

# create csr matrix from given members list
def create_csr_matrix(members, use_weights=False):
    clients = list(set([member['Client1'] for member in members] + [member['Client2'] for member in members]))
    client_index = {client: idx for idx, client in enumerate(clients)}

    row_indices = []
    col_indices = []
    data = []

    for member in members:
        row_indices.append(client_index[member['Client1']])
        col_indices.append(client_index[member['Client2']])
        if use_weights:
            data.append(float(member['duration_minutes']))  # Use duration in minutes as the weight of the edge
        else:
            data.append(1)  # Use 1 for unweighted similarity

    num_clients = len(clients)
    csr = csr_matrix((data, (row_indices, col_indices)), shape=(num_clients, num_clients))

    # Serialize the CSR matrix
    serialized_csr = base64.b64encode(pickle.dumps(csr)).decode('utf-8')
    return serialized_csr

# compare given two csr matrices (each relating to a community) to get similarity score
def compare_weighted_structural_similarity(csr_matrix_1, csr_matrix_2):
    # Deserialize CSR matrices
    csr_1 = pickle.loads(base64.b64decode(csr_matrix_1))
    csr_2 = pickle.loads(base64.b64decode(csr_matrix_2))


    # Align matrix dimensions to the largest size
    max_rows = max(csr_1.shape[0], csr_2.shape[0])
    max_cols = max(csr_1.shape[1], csr_2.shape[1])

    # Pad csr_1 to match max dimensions
    if csr_1.shape[0] < max_rows or csr_1.shape[1] < max_cols:
        csr_1 = vstack([csr_1, csr_matrix((max_rows - csr_1.shape[0], csr_1.shape[1]))]) if csr_1.shape[0] < max_rows else csr_1
        csr_1 = hstack([csr_1, csr_matrix((csr_1.shape[0], max_cols - csr_1.shape[1]))]) if csr_1.shape[1] < max_cols else csr_1

    # Pad csr_2 to match max dimensions
    if csr_2.shape[0] < max_rows or csr_2.shape[1] < max_cols:
        csr_2 = vstack([csr_2, csr_matrix((max_rows - csr_2.shape[0], csr_2.shape[1]))]) if csr_2.shape[0] < max_rows else csr_2
        csr_2 = hstack([csr_2, csr_matrix((csr_2.shape[0], max_cols - csr_2.shape[1]))]) if csr_2.shape[1] < max_cols else csr_2

    # Calculate structural similarity (e.g., using cosine similarity)
    dot_product = csr_1.multiply(csr_2).sum()
    norm_1 = np.sqrt(csr_1.multiply(csr_1).sum())
    norm_2 = np.sqrt(csr_2.multiply(csr_2).sum())
    similarity = dot_product / (norm_1 * norm_2) if norm_1 != 0 and norm_2 != 0 else 0
    return float(similarity)

In [6]:
# Generate datasets
# Initialize Spark session
spark = SparkSession.builder \
  .appName("PhoneCallsCommunityDetection") \
  .master("local[*]") \
  .config("spark.jars.packages", "ch.cern.sparkmeasure:spark-measure_2.12:0.24,graphframes:graphframes:0.8.2-spark3.1-s_2.12") \
  .config("spark.executor.memory", "20G") \
  .config("spark.driver.memory", "50G") \
  .config("spark.executor.memoryOverhead", "1G") \
  .config("spark.default.parallelism", "100") \
  .config("spark.sql.shuffle.partitions", "10") \
  .config("spark.driver.maxResultSize", "2G") \
  .getOrCreate()


# Initialize StageMetrics
stagemetrics = StageMetrics(spark)

# Optional: Set logging level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

# Set a checkpoint directory for Spark
spark.sparkContext.setCheckpointDir("/tmp/spark-checkpoints")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import random
import re

def create_spark_session(app_name="CallDatasetGenerator"):
    """
    Create or get a Spark session for distributed data processing.

    Parameters:
        app_name (str): The name of the Spark application.

    Returns:
        SparkSession: An active Spark session object.
    """
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.shuffle.partitions", "200") \
        .getOrCreate()

def generate_communities(spark, num_communities, community_size_range, density=0.3):
    """
    Generate base communities with client connections based on parameters. Each community will have a defined
    size and connections based on the specified density.

    Parameters:
        spark (SparkSession): Active Spark session.
        num_communities (int): Number of distinct communities to generate.
        community_size_range (tuple): Min and max range for the size (number of clients) in each community.
        density (float): Probability for a connection to exist between two clients within a community.

    Returns:
        DataFrame: A Spark DataFrame with columns [community_id, client1, client2].
    """
    # Create RDD with community sizes for each community ID
    sizes_rdd = spark.sparkContext.parallelize(
        [(i, random.randint(community_size_range[0], community_size_range[1]))
         for i in range(num_communities)]
    )

    # Generate potential client connections within each community
    def generate_connections(row):
        community_id, size = row
        base = community_id * 1000  # Offset to ensure unique client IDs across communities
        connections = []
        for i in range(size):
            for j in range(i + 1, size):
                if random.random() < density:
                    connections.append((community_id, base + i, base + j))
        return connections

    connections_rdd = sizes_rdd.flatMap(generate_connections)
    return spark.createDataFrame(connections_rdd, ["community_id", "client1", "client2"])

def generate_call_times(communities_df, calls_per_connection_range, duration_range, base_time):
    """
    Generate call start and end times for each client connection. Times are formatted as yymmddHHMM strings.

    Parameters:
        communities_df (DataFrame): DataFrame with columns [community_id, client1, client2] representing client pairs.
        calls_per_connection_range (tuple): Min and max range for the number of calls per client connection.
        duration_range (tuple): Min and max range for the duration (in minutes) of each call.
        base_time (datetime): Base datetime for the start time of generated calls.

    Returns:
        DataFrame: A Spark DataFrame with columns [client1, client2, Start_Time, End_Time].
    """
    # Add a random number of calls for each client connection
    calls_df = communities_df.withColumn(
        "num_calls",
        F.expr(f"floor(rand() * ({calls_per_connection_range[1]} - {calls_per_connection_range[0]} + 1)) + {calls_per_connection_range[0]}")
    ).withColumn(
        "call_id", F.monotonically_increasing_id()  # Unique ID for each call
    ).withColumn(
        "calls", F.expr("sequence(1, num_calls)")  # Sequence for each call in connection
    ).select("community_id", "client1", "client2", "call_id", F.explode("calls").alias("call_num"))

    # UDF to generate start and end times for calls with random durations
    def generate_times(row):
        start_time = base_time + timedelta(minutes=random.randint(0, 1440))  # Random minute offset within a day
        duration = random.randint(duration_range[0], duration_range[1])
        end_time = start_time + timedelta(minutes=duration)
        return start_time.strftime('%y%m%d%H%M'), end_time.strftime('%y%m%d%H%M')

    time_udf = F.udf(lambda x: generate_times(x), "struct<Start_Time:string, End_Time:string>")
    calls_df = calls_df.withColumn("call_times", time_udf(F.col("call_id")))
    return calls_df.select("client1", "client2", "call_times.Start_Time", "call_times.End_Time")

def save_dataset(dataset, params, spark):
    """
    Save the generated dataset to CSV with a filename based on the parameters used in generation.

    Parameters:
        dataset (DataFrame): The DataFrame to save, containing generated call data.
        params (dict): Dictionary of parameters used for generation, used in the filename.
        spark (SparkSession): Active Spark session.
    """
    # Construct a filename based on parameters
    filename = "_".join(f"{key}_{value}" for key, value in params.items())
    dataset.write.mode("overwrite").option("header", "true").csv(f"/content/{filename}.csv")
    print(f"Dataset saved as /content/{filename}.csv")

if __name__ == "__main__":
    # Initialize Spark session
    spark = create_spark_session()

    # Example parameters for dataset generation
    num_communities = 10  # Number of distinct communities
    community_size_range = (3, 8)  # Min and max clients per community
    calls_per_connection_range = (1, 5)  # Min and max number of calls per client pair
    duration_range = (30, 180)  # Call duration range in minutes
    density = 0.3  # Probability for connection existence within a community
    base_time = datetime(2024, 1, 1)  # Base datetime for generated calls

    # Step 1: Generate communities
    communities_df = generate_communities(spark, num_communities, community_size_range, density)

    # Step 2: Generate call times based on community structure
    calls_df = generate_call_times(communities_df, calls_per_connection_range, duration_range, base_time)

    # Step 3: Save generated dataset to CSV with parameters in filename
    save_params = {
        "num_communities": num_communities,
        "community_size_range": community_size_range,
        "calls_per_connection_range": calls_per_connection_range,
        "duration_range": duration_range,
        "density": density
    }
    save_dataset(calls_df, save_params, spark)

    # Stop Spark session after completing tasks
    spark.stop()


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import random
import re

def create_spark_session(app_name="CallDatasetGenerator"):
    """
    Create or get a Spark session for distributed data processing.

    Parameters:
        app_name (str): The name of the Spark application.

    Returns:
        SparkSession: An active Spark session object.
    """
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.shuffle.partitions", "200") \
        .getOrCreate()

def generate_communities(spark, num_communities, community_size_range, density=0.3):
    """
    Generate base communities with client connections based on parameters. Each community will have a defined
    size and connections based on the specified density.

    Parameters:
        spark (SparkSession): Active Spark session.
        num_communities (int): Number of distinct communities to generate.
        community_size_range (tuple): Min and max range for the size (number of clients) in each community.
        density (float): Probability for a connection to exist between two clients within a community.

    Returns:
        DataFrame: A Spark DataFrame with columns [community_id, client1, client2].
    """
    # Create RDD with community sizes for each community ID
    sizes_rdd = spark.sparkContext.parallelize(
        [(i, random.randint(community_size_range[0], community_size_range[1]))
         for i in range(num_communities)]
    )

    # Generate potential client connections within each community
    def generate_connections(row):
        community_id, size = row
        base = community_id * 1000  # Offset to ensure unique client IDs across communities
        connections = []
        for i in range(size):
            for j in range(i + 1, size):
                if random.random() < density:
                    connections.append((community_id, base + i, base + j))
        return connections

    connections_rdd = sizes_rdd.flatMap(generate_connections)
    return spark.createDataFrame(connections_rdd, ["community_id", "client1", "client2"])

def generate_call_times(communities_df, calls_per_connection_range, duration_range, base_time):
    """
    Generate call start and end times for each client connection. Times are formatted as yymmddHHMM strings.

    Parameters:
        communities_df (DataFrame): DataFrame with columns [community_id, client1, client2] representing client pairs.
        calls_per_connection_range (tuple): Min and max range for the number of calls per client connection.
        duration_range (tuple): Min and max range for the duration (in minutes) of each call.
        base_time (datetime): Base datetime for the start time of generated calls.

    Returns:
        DataFrame: A Spark DataFrame with columns [client1, client2, Start_Time, End_Time].
    """
    # Add a random number of calls for each client connection
    calls_df = communities_df.withColumn(
        "num_calls",
        F.expr(f"floor(rand() * ({calls_per_connection_range[1]} - {calls_per_connection_range[0]} + 1)) + {calls_per_connection_range[0]}")
    ).withColumn(
        "call_id", F.monotonically_increasing_id()  # Unique ID for each call
    ).withColumn(
        "calls", F.expr("sequence(1, num_calls)")  # Sequence for each call in connection
    ).select("community_id", "client1", "client2", "call_id", F.explode("calls").alias("call_num"))

    # UDF to generate start and end times for calls with random durations
    def generate_times(row):
        start_time = base_time + timedelta(minutes=random.randint(0, 1440))  # Random minute offset within a day
        duration = random.randint(duration_range[0], duration_range[1])
        end_time = start_time + timedelta(minutes=duration)
        return start_time.strftime('%y%m%d%H%M'), end_time.strftime('%y%m%d%H%M')

    time_udf = F.udf(lambda x: generate_times(x), "struct<Start_Time:string, End_Time:string>")
    calls_df = calls_df.withColumn("call_times", time_udf(F.col("call_id")))
    return calls_df.select("client1", "client2", "call_times.Start_Time", "call_times.End_Time")

def save_dataset(dataset, params, spark):
    """
    Save the generated dataset to CSV with a filename based on the parameters used in generation.

    Parameters:
        dataset (DataFrame): The DataFrame to save, containing generated call data.
        params (dict): Dictionary of parameters used for generation, used in the filename.
        spark (SparkSession): Active Spark session.
    """
    # Construct a filename based on parameters
    filename = "_".join(f"{key}_{value}" for key, value in params.items())
    dataset.write.mode("overwrite").option("header", "true").csv(f"/content/{filename}.csv")
    print(f"Dataset saved as /content/{filename}.csv")

if __name__ == "__main__":
    # Initialize Spark session
    spark = create_spark_session()

    # Example parameters for dataset generation
    num_communities = 10  # Number of distinct communities
    community_size_range = (3, 8)  # Min and max clients per community
    calls_per_connection_range = (1, 5)  # Min and max number of calls per client pair
    duration_range = (30, 180)  # Call duration range in minutes
    density = 0.3  # Probability for connection existence within a community
    base_time = datetime(2024, 1, 1)  # Base datetime for generated calls

    # Step 1: Generate communities
    communities_df = generate_communities(spark, num_communities, community_size_range, density)

    # Step 2: Generate call times based on community structure
    calls_df = generate_call_times(communities_df, calls_per_connection_range, duration_range, base_time)

    # Step 3: Save generated dataset to CSV with parameters in filename
    save_params = {
        "num_communities": num_communities,
        "community_size_range": community_size_range,
        "calls_per_connection_range": calls_per_connection_range,
        "duration_range": duration_range,
        "density": density
    }
    save_dataset(calls_df, save_params, spark)

    # Stop Spark session after completing tasks
    spark.stop()

Dataset saved as /content/num_communities_10_community_size_range_(3, 8)_calls_per_connection_range_(1, 5)_duration_range_(30, 180)_density_0.3.csv


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import random
import re
from google.colab import files  # Import for downloading


def create_spark_session(app_name="CallDatasetGenerator"):
    """
    Create or get a Spark session for distributed data processing.

    Parameters:
        app_name (str): The name of the Spark application.

    Returns:
        SparkSession: An active Spark session object.
    """
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.shuffle.partitions", "200") \
        .getOrCreate()

def generate_communities(spark, num_communities, community_size_range, density=0.3):
    """
    Generate base communities with client connections based on parameters. Each community will have a defined
    size and connections based on the specified density.

    Parameters:
        spark (SparkSession): Active Spark session.
        num_communities (int): Number of distinct communities to generate.
        community_size_range (tuple): Min and max range for the size (number of clients) in each community.
        density (float): Probability for a connection to exist between two clients within a community.

    Returns:
        DataFrame: A Spark DataFrame with columns [community_id, client1, client2].
    """
    # Create RDD with community sizes for each community ID
    sizes_rdd = spark.sparkContext.parallelize(
        [(i, random.randint(community_size_range[0], community_size_range[1]))
         for i in range(num_communities)]
    )

    # Generate potential client connections within each community
    def generate_connections(row):
        community_id, size = row
        base = community_id * 1000  # Offset to ensure unique client IDs across communities
        connections = []
        for i in range(size):
            for j in range(i + 1, size):
                if random.random() < density:
                    connections.append((community_id, base + i, base + j))
        return connections

    connections_rdd = sizes_rdd.flatMap(generate_connections)
    return spark.createDataFrame(connections_rdd, ["community_id", "client1", "client2"])

def generate_call_times(communities_df, calls_per_connection_range, duration_range, base_time):
    """
    Generate call start and end times for each client connection. Times are formatted as yymmddHHMM strings.

    Parameters:
        communities_df (DataFrame): DataFrame with columns [community_id, client1, client2] representing client pairs.
        calls_per_connection_range (tuple): Min and max range for the number of calls per client connection.
        duration_range (tuple): Min and max range for the duration (in minutes) of each call.
        base_time (datetime): Base datetime for the start time of generated calls.

    Returns:
        DataFrame: A Spark DataFrame with columns [client1, client2, Start_Time, End_Time].
    """
    # Add a random number of calls for each client connection
    calls_df = communities_df.withColumn(
        "num_calls",
        F.expr(f"floor(rand() * ({calls_per_connection_range[1]} - {calls_per_connection_range[0]} + 1)) + {calls_per_connection_range[0]}")
    ).withColumn(
        "call_id", F.monotonically_increasing_id()  # Unique ID for each call
    ).withColumn(
        "calls", F.expr("sequence(1, num_calls)")  # Sequence for each call in connection
    ).select("community_id", "client1", "client2", "call_id", F.explode("calls").alias("call_num"))

    # UDF to generate start and end times for calls with random durations
    def generate_times(row):
        start_time = base_time + timedelta(minutes=random.randint(0, 1440))  # Random minute offset within a day
        duration = random.randint(duration_range[0], duration_range[1])
        end_time = start_time + timedelta(minutes=duration)
        return start_time.strftime('%y%m%d%H%M'), end_time.strftime('%y%m%d%H%M')

    time_udf = F.udf(lambda x: generate_times(x), "struct<Start_Time:string, End_Time:string>")
    calls_df = calls_df.withColumn("call_times", time_udf(F.col("call_id")))
    return calls_df.select("client1", "client2", "call_times.Start_Time", "call_times.End_Time")

def save_dataset(dataset, params, spark):
    """
    Save the generated dataset to CSV with a filename based on the parameters used in generation.

    Parameters:
        dataset (DataFrame): The DataFrame to save, containing generated call data.
        params (dict): Dictionary of parameters used for generation, used in the filename.
        spark (SparkSession): Active Spark session.
    """
    # Construct a filename based on parameters
    filename = "_".join(f"{key}_{value}" for key, value in params.items())
    dataset.write.mode("overwrite").option("header", "true").csv(f"/content/{filename}.csv")
    print(f"Dataset saved as /content/{filename}.csv")

# Function to delete all generated datasets
def delete_generated_datasets():
    deleted_files = []
    for file in os.listdir():
        if file.startswith(prefix):
            os.remove(file)
            deleted_files.append(file)
    if deleted_files:
        print("Deleted the following files:")
        for file in deleted_files:
            print(file)
    else:
        print("No generated datasets found to delete.")


# Define a function to generate a dataset based on given parameters
def generate_dataset(num_samples, num_features, noise_level):
    data = np.random.rand(num_samples, num_features) + noise_level
    df = pd.DataFrame(data, columns=[f'feature_{i}' for i in range(num_features)])
    return df

# Prefix for identifying generated datasets
prefix = "generated_"

# List of settings (adjust to your needs)
settings = [
    {'num_samples': 100, 'num_features': 5, 'noise_level': 0.1},
    # {'num_samples': 200, 'num_features': 6, 'noise_level': 0.2},
    # {'num_samples': 300, 'num_features': 7, 'noise_level': 0.3},
    # {'num_samples': 400, 'num_features': 8, 'noise_level': 0.4},
    # {'num_samples': 500, 'num_features': 9, 'noise_level': 0.5}
]

# Loop over settings, generate datasets, and save them
for i, setting in enumerate(settings, start=1):
    dataset = generate_dataset(**setting)
    filename = f"{prefix}dataset_{i}_samples{setting['num_samples']}_features{setting['num_features']}_noise{setting['noise_level']}.csv"
    dataset.to_csv(filename, index=False)

    # Print information about the created dataset
    print(f"Created dataset with {setting['num_samples']} samples and {setting['num_features']} features, "
          f"noise level {setting['noise_level']}. Saved as {filename}.")
    print(f"Number of samples in dataset: {len(dataset)}")

    # Download the CSV file if running in Colab
    files.download(filename)

# Uncomment the line below to delete all generated datasets
# delete_generated_datasets()

print("All datasets have been generated and saved.")

Created dataset with 100 samples and 5 features, noise level 0.1. Saved as generated_dataset_1_samples100_features5_noise0.1.csv.
Number of samples in dataset: 100


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All datasets have been generated and saved.


In [12]:
%rm -rf "/content/num_communities_10_community_size_range_(3, 8)_calls_per_connection_range_(1, 5)_duration_range_(30, 180)_density_0.3.csv"

In [ ]:
# Read the
@track_stage("Stage 1: Reading the calls dataset")
def read_csv_to_dataframe(file_path= 'toy_dataset.csv'):
  """
  Read dataset from given path into a Spark DataFrame.
  Parameters:
    -----------
    file_path : str
        The name of the given dataset (unigrams or bigrams or both).

    Returns:
    --------
    df_dataset : DataFrame
        A DataFrame of calls with the given dataset info.
  """
  df_dataset = spark.read.csv(file_path, header=True, inferSchema=True)

  # convert start - end times to duration
  # 1st Register the UDFs in Spark
  calculate_duration_minutes_udf = udf(calculate_duration_minutes, DoubleType())
  calculate_duration_string_udf = udf(calculate_duration_string, StringType())

  # 2nd use udfs to add columns for duration in minutes and DDHHMM format
  df_dataset = df_dataset.withColumn('duration_minutes', calculate_duration_minutes_udf(col('Start_Time'), col('End_Time')))
  df_dataset = df_dataset.withColumn('duration_DDHHMM', calculate_duration_string_udf(col('Start_Time'), col('End_Time')))

  print("The following dataframe has been read from the CSV file:")
  df_dataset.show()
  return df_dataset

@track_stage("Stage 2: Preprocessing and creating the graph")
def create_graph_from_dataframe(df_dataset):
  """
  Create graph in GraphFrame from the calls in the current dataset.
  Parameters:
    -----------
    df_dataset : DataFrame
        A DataFrame of calls with the given dataset info.

    Returns:
    --------
    df_dataset : DataFrame
        A DataFrame of calls with the given dataset info.
  """

  # Create Graph using GraphFrames for community detection
  vertices = df_dataset.selectExpr("Client1 as id").union(df_dataset.selectExpr("Client2 as id")).distinct()
  edges = df_dataset.selectExpr("Client1 as src", "Client2 as dst", "duration_minutes as weight")

  # Cache vertices and edges
  vertices.cache()
  edges.cache()

  # Create a GraphFrame
  g = GraphFrame(vertices, edges)

  # Find connected components (communities) using GraphFrames
  connected_components_result = g.connectedComponents()

  # Create a mapping from original community IDs to sequential ones
  community_mapping = connected_components_result.select("component").distinct() \
      .orderBy("component") \
      .withColumn("new_id", row_number().over(Window.orderBy("component"))) \
      .cache()

  # Join the result (community IDs) with the original dataframe and map to new sequential IDs
  df_with_communities = df_dataset.join(connected_components_result, df_dataset['Client1'] == connected_components_result['id'], 'inner') \
      .join(community_mapping, connected_components_result['component'] == community_mapping['component'], 'inner') \
      .drop(connected_components_result['id']) \
      .drop(community_mapping['component']) \
      .withColumnRenamed('new_id', 'community_id')

  # Calculate the number of unique clients (community size) per community
  community_sizes = df_with_communities.select("community_id", "Client1").union(df_with_communities.select("community_id", "Client2")) \
      .distinct() \
      .groupBy("community_id").agg(countDistinct("Client1").alias("community_size"))

  # Merge the community sizes into the main DataFrame
  df_final = df_with_communities.join(community_sizes, 'community_id')

  # Get list of tuples for each community member by considering both Client1 and Client2
  community_members = df_final.select("community_id", "Client1", "Client2", "duration_DDHHMM", "duration_minutes") \
      .distinct() \
      .groupBy("community_id") \
      .agg(collect_list(struct(col("Client1"),
                            col("Client2"),
                            col("duration_DDHHMM"),
                            col("duration_minutes"))).alias("members")) \
      .orderBy("community_id")

  # Show the final DataFrame with community IDs, duration, and community sizes
  print("\nFinal DataFrame with Sequential Community IDs:")
  df_final.select('Client1',
                  'Client2',
                  'duration_DDHHMM',
                  'duration_minutes',
                  'community_id',
                  'community_size') \
      .orderBy("community_id") \
      .show()

  # Show the list of community members as tuples
  print("\nCommunity Members with Sequential IDs:")
  community_members.show(truncate=False)

  # Save results to CSV files
  # Save the main analysis results
  df_final.select('Client1',
                  'Client2',
                  'duration_DDHHMM',
                  'duration_minutes',
                  'community_id',
                  'community_size') \
      .orderBy("community_id") \
      .write.mode("overwrite").option("header", "true")
      .csv(f"{dataset_name}_community_analysis_results")

  # Save community members in a flattened format
  df_final.select('community_id',
                  'Client1',
                  'Client2',
                  'duration_DDHHMM',
                  'duration_minutes') \
      .distinct() \
      .orderBy("community_id") \
      .write.mode("overwrite").option("header", "true") \
      .csv(f"{dataset_name}_community_members_results")

  # Optionally, if you want to save additional community statistics
  community_stats = df_final.groupBy('community_id') \
      .agg(
          countDistinct('Client1', 'Client2').alias('unique_members'),
          count('*').alias('total_calls'),
          sum('duration_minutes').alias('total_duration_minutes'),
          avg('duration_minutes').alias('avg_call_duration'),
          percentile_approx('duration_minutes', 0.25).alias('duration_25th_percentile'),
          percentile_approx('duration_minutes', 0.5).alias('median_call_duration'),
          percentile_approx('duration_minutes', 0.75).alias('duration_75th_percentile')
      ) \
      .orderBy('community_id')

  community_stats.write.mode("overwrite").option("header", "true").csv("community_statistics_results")

  print("This is the community stats:")
  community_stats.show(truncate=False)
  return df_final, community_members, community_stats

# Create CSR adjacency matrices for each community and serialize them
@track_stage("Stage 3: Creating CSR matrices")
def format_members_to_csr_matrix(community_members):
  """
  Create CSR adjacency matrices for each community and serialize them.

  Parameters:
    community_members: Dataframe
    A dataframe of a specific community's members
  """
  # Convert the collected list of Row objects to a list of dictionaries before passing to UDF
  schema = StructType([
      StructField("Client1", StringType(), True),
      StructField("Client2", StringType(), True),
      StructField("duration_DDHHMM", StringType(), True),
      StructField("duration_minutes", DoubleType(), True)
  ])
  convert_members_udf = udf(lambda members: [member.asDict() for member in members], ArrayType(schema))
  community_members = community_members.withColumn("members_dict", convert_members_udf(col("members")))
  #Register UDF to create and serialize CSR matrices (both unweighted and weighted)
  create_csr_unweighted_udf = udf(lambda members: create_csr_matrix(members, use_weights=False), StringType())
  create_csr_weighted_udf = udf(lambda members: create_csr_matrix(members, use_weights=True), StringType())

  # Add CSR matrix representations (unweighted and weighted) to each community
  community_members = community_members.withColumn("csr_matrix_unweighted", create_csr_unweighted_udf(col("members_dict")))
  community_members = community_members.withColumn("csr_matrix_weighted", create_csr_weighted_udf(col("members_dict")))

  community_members.show(truncate=False)

  # Print some information about the matrix
  # print(f"CSR Matrix shape: {csr_matrix_result.shape}")
  # print(f"Number of non-zero elements: {csr_matrix_result.nnz}")
  # pretty_print_csr_matrix(csr_matrix_result)

  return community_members

@track_stage("Stage 4: Calculate similarities between communities")
def calculate_similarities(community_members):
  """
  Comparing CSR matrices to detect similarity
  """

  # Register UDF to compare structural similarity
  compare_structural_similarity_udf = udf(lambda csr_1, csr_2: compare_weighted_structural_similarity(csr_1, csr_2), DoubleType())
  compare_weighted_similarity_udf = udf(lambda csr_1, csr_2: compare_weighted_structural_similarity(csr_1, csr_2), DoubleType())

  # Cross join to compare each pair of communities and calculate both similarities
  cross_joined = community_members.alias("a").crossJoin(community_members.alias("b")) \
      .filter(col("a.community_id") < col("b.community_id")) \
      .withColumn("unweighted_similarity_score", compare_structural_similarity_udf(col("a.csr_matrix_unweighted"), col("b.csr_matrix_unweighted"))) \
      .withColumn("weighted_similarity_score", compare_weighted_similarity_udf(col("a.csr_matrix_weighted"), col("b.csr_matrix_weighted")))

  # Add combined similarity score (50/50 importance)
  cross_joined = cross_joined.withColumn("combined_similarity_score",
                                        0.5 * col("unweighted_similarity_score") + 0.5 * col("weighted_similarity_score"))

  # Show the similarity scores between communities
  # Rename and select columns to create a new DataFrame for writing
  df_to_export = cross_joined.select(
      col("a.community_id").alias("community_id_1"),
      col("b.community_id").alias("community_id_2"),
      round(col("unweighted_similarity_score"), 2).alias("unweighted_similarity_score"),
      round(col("weighted_similarity_score"), 2).alias("weighted_similarity_score"),
      round(col("combined_similarity_score"), 2).alias("combined_similarity_score")
  )

  # Display the DataFrame
  print("these are the groups:")
  df_to_export.orderBy(["community_id_1", "community_id_2"]).show(truncate=False)

  # Write the new DataFrame to CSV
  print("exporting to csv file")
  df_to_export.write.mode("overwrite").option("header", "true").csv("groups_found")

  return cross_joined

In [ ]:
# read dataset dataframe, and iterate over each one to create communities and form groups
df_datasets = pd.read_csv("dataset_metadata.csv")
print("These are the found datasets")
df_datasets.head(10)

# set global params
clear_csv = False
dataset_file_path = "toy_dataset.csv"
dataset_name = "toy_dataset"

for i, dataset in df_datasets.iterrows():
  print(f"Starting to process {i+1} dataset with the following params: \n{dataset}")
  # step 1 - read the dataset
  dataset_file_path = dataset["csv_filename"]
  # Get the base name from the path (e.g., 'file.txt' from '/path/to/file.txt')
  basename =  os.path.basename(filepath)
  # Split the filename and extension
  dataset_name = os.path.splitext(basename)[0]
  clear_csv = i == 0 # only clear the if this is the 1st dataset
  df_dataset = spark.read.csv(dataset_file_path, header=True, inferSchema=True)
  df_dataset.show(20)
  df_dataset = read_csv_to_dataframe(dataset_file_path)

  # step 2 - preprocess (convert to duartion in min, create grpah, and find commutnies)
  df_final, community_members, community_stats = create_graph_from_dataframe(df_dataset)

  # step 3 - create CSR matrix for each communite
  csr_matrix_result = format_members_to_csr_matrix(community_members)

  # step 4 - calculate similarities between communties for find groups
  cross_joined = calculate_similarities(csr_matrix_result)

# # end the current spark session
# spark.stop()

These are the found datasets
Starting to process 1 dataset with the following params: 
num_communities                                               5
community_size_range                                     (3, 8)
calls_per_connection_range                               (1, 3)
duration_range                                        (30, 180)
num_similar_communities_range                            (2, 4)
csv_filename                     call_data_5_38_13_30180_24.csv
Name: 0, dtype: object
+-------+-------+----------+----------+
|Client1|Client2|Start_Time|  End_Time|
+-------+-------+----------+----------+
|   1000|   1001|2401012050|2312311412|
|   1000|   1003|2401012340|2401010212|
|   1001|   1007|2401011134|2312311424|
|   1004|   1006|2401012101|2401011242|
|   2000|   2001|2312310712|2401012221|
|   2000|   2001|2401010117|2401011326|
|   2000|   2001|2401012224|2401011439|
|   2000|   2005|2312311754|2401012320|
|   2000|   2007|2401012014|2312311145|
|   2001|   2002|231231211

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")



Final DataFrame with Sequential Community IDs:
+-------+-------+---------------+----------------+------------+--------------+
|Client1|Client2|duration_DDHHMM|duration_minutes|community_id|community_size|
+-------+-------+---------------+----------------+------------+--------------+
|   1000|   1001|         -21722|         -1838.0|           1|             4|
|   1000|   1003|         -10232|         -1288.0|           1|             4|
|   1001|   1007|         -10250|         -1270.0|           1|             4|
|   1004|   1006|         -11541|          -499.0|           2|             2|
|   2000|   2001|         011509|          2349.0|           3|             5|
|   2000|   2001|         001209|           729.0|           3|             5|
|   2000|   2001|         -11615|          -465.0|           3|             5|
|   2000|   2005|         010526|          1766.0|           3|             5|
|   2000|   2007|         -21531|         -1949.0|           3|             5|
|   

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 32870)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(r

Py4JError: An error occurred while calling o6576.run

In [ ]:
# plot the resources usgae of current dataset
df_monitor = spark.read.csv("stage_metrics",  header=True)
df_monitor = df_monitor.orderBy('stage_name')
print(f"count: {df_monitor.count()}")
df_monitor.show(truncate=False)

# Convert the Spark DataFrame to a Pandas DataFrame for plotting
pdf_monitor = df_monitor.toPandas()

# Select the columns of interest for plotting
columns_of_interest = ['stage_name', 'numTasks', 'stageDuration', 'peakExecutionMemory', 'executorCpuTime']
pdf_plot = pdf_monitor[columns_of_interest].copy()  # Create a copy to avoid the warning

# Format memory and time measurements using .loc
pdf_plot.loc[:, 'peakExecutionMemory'] = pdf_plot['peakExecutionMemory'].astype(float) / (1024**3)  # Bytes to GB
pdf_plot.loc[:, 'stageDuration'] = pdf_plot['stageDuration'].astype(float) / (1000 * 60)  # ms to minutes
pdf_plot.loc[:, 'executorCpuTime'] = pdf_plot['executorCpuTime'].astype(float) / (1000 * 60)  # ms to minutes
pdf_plot.loc[:, 'numTasks'] = pdf_plot['numTasks'].astype(int)

# Extract stage numbers from stage_name
pdf_plot['stage_number'] = pdf_plot['stage_name'].apply(lambda x: int(re.search(r'\d+', x).group()))

# Melt the DataFrame for easier plotting with Seaborn
pdf_plot_melted = pd.melt(pdf_plot, id_vars=['stage_name', 'stage_number'], var_name='Metric', value_name='Value')

fig = plt.figure(figsize=(10, 8))
# Create the plot with facets, stage numbers as x-axis, and legend with full stage names
g = sns.FacetGrid(pdf_plot_melted, col='Metric',
                   height=6, aspect=1.5,
                  col_wrap=2, sharey=False, sharex=False)
g.map(sns.barplot, 'stage_number', 'Value', palette='hls', hue='stage_name',
      data=pdf_plot_melted, dodge=False)  # Pass data argument
g.set_xticklabels(pdf_plot['stage_number'].unique(), size=16)
g.set_titles("{col_name}", size=18)
g.fig.suptitle('Spark Stage Metrics', y=1.02, size=18)
g.add_legend(loc='upper right', bbox_to_anchor=(1.2, 0.92))
g.legend.set_title('Stage Names', prop={'weight': 'bold', 'size': 22})  # Add title and format it
for text in g.legend.get_texts():
    text.set_fontsize(18)  # Legend label font size# Set y-axis labels with units and add x-axis label
for ax in g.axes.flat:
    metric = ax.get_title()
    if metric == 'peakExecutionMemory':
        ax.set_ylabel('Peak Execution Memory (GB)', size=18)
    elif metric in ('stageDuration', 'executorCpuTime'):
        ax.set_ylabel('Time (minutes)', size=18)
    else:
        ax.set_ylabel(metric, size=18)

    ax.set_xlabel('Stage Number', size=18)  # Change x-axis label to "Stage Number"

g.set_yticklabels(fontsize=18)
plt.tight_layout()
plt.show()